<!-- Autoheader begin -->
<hr/>
<div id="navtitle_1_1_py" style="text-align:center; font-size:16px">I.1 Population Inversion in a Two-Level-System</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_1_2_lambda.ipynb">next notebook $\rightarrow$</a><br>
        <a href="py_exercise_1_2_lambda.ipynb" style="font-size:13px">I.2 Population Transfer in a Three-Level-System with STIRAP</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_2_1_TLS.ipynb" style="font-size:13px">II.1 Population Inversion in a Two-Level-System using Parameter Optimization</a><br>
        <a href="py_exercise_2_1_TLS.ipynb">$\downarrow$ next part $\downarrow$</a>
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Julia/jl_exercise_1_1_TLS.ipynb">👉 Julia version</a></div>

---
<!-- Autoheader end -->

# Population Inversion in a Two-Level-System

The purpose of this notebook is to introduce you to the framework of Jupyter notebooks and to demonstrate how they can be used to simulate a simple quantum system - a two-level system. Jupyter notebooks allow to create and share interactive documents including live code and equations and to access them via a web browser. They can work with many programming languages running the calculations in the background - here we use Python. The goal of this notebook is to allow you to gain familiarity with a typical workflow in a Jupyter notebook while learning about the paradigmatic example of light-matter interaction: "Rabi cycling" in a two-level system. Rabi cycling is a term describing the periodic excitation and de-excitation due to coherent interaction with a light field. Working on this example will also allow you to get to know some useful Python packages for the numerical description and simulation of quantum systems.

## How to use this notebook

*You can evaluate all cells marked with `[n]:` by* **selecting it and hitting
SHIFT+ENTER** *or the play button in the top panel.*

Just go through the notebook and evaluate the cells one after another. You
can also change the cell to play around with the values and reevaluate it. If
you do so, make sure to evaluate all the cells that rely on the one you
changed. Have fun!

## Physical background

In this exercise we simulate the interaction of a two-level-system with a
laser pulse.

The Hamiltonian of the two-level-system is defined as

$$
\hat{H}(t) = \hat{H}_0 + \hat{H}_1(t)
\;,
$$

where $\hat{H}_0$ is the time-independent Hamiltonian of the system and
$\hat{H}_1(t)=E(t) \, \hat{V}$ describes the interaction with the field $E(t)$.

Choosing the eigenstates of $\hat{H}_0$ as a basis, $\{|0⟩, |1⟩\}$, we
can represent the system Hamiltonian by

$$
\hat{H}_0 = -\frac{\omega}{2}
\begin{pmatrix}
1 & 0 \\
0 & -1
\end{pmatrix}
$$

and the interaction operator $\hat{V}$ by

$$
\hat{V} = -\mu_{01}
\begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix}
\;.
$$

Here, $\omega>0$ is the energy splitting between the two levels and
$\mu_{01}$ is the transition matrix element.

As you will find out by numerically simulating this system below, driving the
system with a field $E(t)$ with a suitable frequency $\omega$ induces
resonant Rabi cycling, where the population perfectly cycles between the two
levels.

## Selected Python packages

The most important Python packages for numerics and data analysis are
[NumPy](https://numpy.org/doc/stable/user/whatisnumpy.html) and
[SciPy](https://docs.scipy.org/doc/scipy/reference/). NumPy provides you with
the basic data structures (i.e. multidimensional arrays), functions for array
manipulation (reshaping, summation etc.), elementary mathematical functions
(`sqrt`, `exp`, `cos` etc.) and basic linear algebra (diagonalization etc.).

Because `numpy` is so central for numerics in Python, it is customary to
import it with an abbreviated name

In [ ]:
import numpy as np

You may also import symbols and functions from `numpy` for ease of use.

In [ ]:
from numpy import pi, sqrt, exp, sin, cos

SciPy adds more specialized functionality like signal processing, regression,
numerical integration, differential equation solvers and much more. We will
not use it in this notebook, but in general, it should be part of your
toolbox.

The Python package [Matplotlib](https://matplotlib.org/) is the base package
for 2d data visualisation. It closely resembles the plotting syntax from
Matlab. It is customary to load the high-level `pyplot` interface from
`matplotlib` as

In [ ]:
import matplotlib.pyplot as plt

[QuTiP](https://qutip.org/) is a great package for simulating quantum
mechanical systems. It is developed for simulating closed and open quantum
systems with a focus on quantum information processing. In particular, it
defines a data structure for quantum objects (`qutip.Qobj`) and comes with a
solver for the Schrödinger and Master equation (`qutip.mesolve()`).

In [ ]:
import qutip

## Let's start!

We start with defining the time interval for the propagation. For numerical
calculations, we need to represent the time interval by a grid with a finite
amount of grid points.

Let the time grid start at `t_start=0` and end at `t_stop=50` with a total
amount of `nt=10000` grid points. We can create such a grid with NumPy's
`np.linspace()` function:

In [ ]:
t_start = 0
t_stop = 50
nt = 10000
t = np.linspace(t_start, t_stop, nt)

## The model

Now we need to define the individual parts of the Hamiltonian. For the
simulation use the following parameters:

In [ ]:
ω = 10
μ_01 = 1

Note that Python has some support for unicode. In the notebooks, we will use it to
display Greek letters for variable names and operators.

In Jupyter notebooks, you can type Greek letters by using LaTeX macros and pressing the
Tab key to convert it. E.g. `\omega[tab]` will be converted to `ω`, or `\mu[tab]` becomes `μ`.

We also need the matrix for the time independent Hamiltonian $\hat{H}_0$ ...

In [ ]:
H0 = -ω/2 * qutip.Qobj(
    np.array([[1,  0],
              [0, -1]])
)

display(H0) # shows the result

... and the matrix for the interaction operator $\hat{V}$

In [ ]:
V = -μ_01 * qutip.Qobj(
    np.array([[0, 1],
              [1, 0]])
)

display(V) # shows the result

Next, we need to define the electric field `E` on the time grid `t`.

We assume a Gaussian shaped pulse,

$$
E(t) = E_0 \, e^{-(t-t_0)^2 / (2 \tau^2)} \, \cos(\omega_l (t-t_0) + \phi)
$$

with the following parameters: $\omega_l=\omega$, $\phi=0$, $E_0=0.2$ and
$t_0=25$.

When choosing a value for the pulse duration $\tau$ one needs to be careful not to choose a duration larger than $7.5$, in order to ensure that the pulse fits completely onto our time grid.

In [ ]:
t_0 = 25
ω_l = ω
E0 = 0.2
ϕ = 0
τ = 2.5

E = E0 * cos(ω_l * (t - t_0) + ϕ) * exp(-((t - t_0) ** 2) / (2 * τ ** 2))

Now we collect everything together and assemble the total Hamiltonian of our
system. The QuTiP package wants us to store the Hamiltonian as nested list in
the following form:

In [ ]:
H = [H0, [V, E]]

As a last step in setting up the model, we define the two states.

In [ ]:
ψ0 = qutip.Qobj(np.array([1, 0]))  # State |0⟩
ψ1 = qutip.Qobj(np.array([0, 1]))  # State |1⟩

display(ψ0, ψ1)  # shows the result

## Propagation and results

Before we can start with the propagation, we first need to define the
observables that we are interested in. For the present case, we are
interested in the population dynamics. To track the population of the two
levels, we define the projectors $\hat{P}_{i} = |i⟩⟨i|$.

In [ ]:
proj0 = ψ0 * ψ0.dag()
proj1 = ψ1 * ψ1.dag()

Having them set up, we can use QuTiP's **m**aster **e**quation **solve**r
function to obtain the dynamics over time.
As initial state of our simulation we choose the ground state, $|0⟩$.

In [ ]:
output = qutip.mesolve(H, ψ0, t, e_ops=[proj0, proj1])

Now let's plot the population dynamics:

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, np.abs(E) / np.abs(E).max(), '-', color='lightgrey', label='|E|')
ax.plot(t, output.expect[0], '-', label=r'$|0\rangle$')
ax.plot(t, output.expect[1], '--', label=r'$|1\rangle$')
ax.set_xlim(t.min(), t.max())
ax.set_xlabel('Time')
ax.set_ylabel('Population')
ax.legend()
plt.show()

Play with the pulse parameters and observe how this affects the population
dynamics. Can you find a combination of parameters that produces a complete
population inversion? What do you need to obtain a full Rabi cycle? What
happens if you change the frequency of the laser pulse?

## Next steps

Continue with [Exercise I.2](py_exercise_1_2_lambda.ipynb) to learn about about slightly more advanced light-matter-interaction in a three-level system, or with [Exercise I.3](py_exercise_1_3_chirp.ipynb) about the interaction of the same two-level-system with a *chirped* laser pulse. [Exercise II.1](py_exercise_2_1_TLS.ipynb) explains how to find the proper parameters to achieve the population inversion discussed above with a gradient-free optimization. [Exercise III.1](py_exercise_3_1_TLS.ipynb) does the same with a gradient-based approach (Krotov's method and GRAPE).

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_1_1_py)
<!-- Autofooter end -->